<a href="https://colab.research.google.com/github/wahome-cloud/vehicle-damage-detection-and-repair-cost-estimation/blob/main/Final2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone https://github.com/facebookresearch/detectron2.git
%cd detectron2
!pip install -e .

Cloning into 'detectron2'...
remote: Enumerating objects: 15832, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 15832 (delta 27), reused 12 (delta 12), pack-reused 15780 (from 2)
Receiving objects: 100% (15832/15832), 6.40 MiB | 19.10 MiB/s, done.
Resolving deltas: 100% (11534/11534), done.
/content/detectron2
Obtaining file:///content/detectron2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.

In [ ]:
import numpy as np
import cv2
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo

# Load Severity Model
cfg_severity = get_cfg()
cfg_severity.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg_severity.MODEL.ROI_HEADS.NUM_CLASSES = 3  # Number of severity classes
cfg_severity.MODEL.WEIGHTS = "/content/drive/MyDrive/severitymodel/severity_model_final.pth"
cfg_severity.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor_severity = DefaultPredictor(cfg_severity)

# Load Car Part Model
cfg_parts = get_cfg()
cfg_parts.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg_parts.MODEL.ROI_HEADS.NUM_CLASSES = 5  # Number of car part classes
cfg_parts.MODEL.WEIGHTS = "/content/drive/MyDrive/TrainedModels/model_finalee.pth"
cfg_parts.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor_parts = DefaultPredictor(cfg_parts)

# Run predictions on an image
image_path = "/content/drive/MyDrive/52.jpg"
image = cv2.imread(image_path)

# Severity Predictions
severity_outputs = predictor_severity(image)
severity_instances = severity_outputs["instances"].to("cpu")
severity_classes = severity_instances.pred_classes.numpy()
severity_scores = severity_instances.scores.numpy()

# Car Part Predictions
part_outputs = predictor_parts(image)
part_instances = part_outputs["instances"].to("cpu")
part_classes = part_instances.pred_classes.numpy()
part_scores = part_instances.scores.numpy()

# Define class names
severity_class_names = ["minor", "moderate", "severe"]
part_class_names = ["headlamp", "rear_bumper", "door", "hood", "front_bumper"]

# Combine results with the highest confidence per part
combined_results = {}
tolerance = 1e-5  # Small value to handle floating-point imprecision

for i, part_class in enumerate(part_classes):
    part_name = part_class_names[part_class]
    part_score = part_scores[i]

    for j, severity_class in enumerate(severity_classes):
        severity_label = severity_class_names[severity_class]
        severity_score = severity_scores[j]

        # Logic to assign the highest confidence severity to each part
        if part_name not in combined_results:
            combined_results[part_name] = {
                "severity": severity_label,
                "confidence": part_score * severity_score,  # Combined confidence
            }
        else:
            current_confidence = combined_results[part_name]["confidence"]
            new_confidence = part_score * severity_score

            if new_confidence > current_confidence + tolerance:
                combined_results[part_name] = {
                    "severity": severity_label,
                    "confidence": new_confidence,
                }



# Format final results
# Format final results
final_results = [f"{part} {data['severity']}" for part, data in combined_results.items()]
print("Final Results:", final_results)

# Calculate total price
prices = {
    "door severe": 13000,
    "door moderate": 4000,
    "door minor": 2000,
    "headlamp severe": 20000,
    "headlamp moderate": 8000,
    "headlamp minor": 2500,
    "rear_bumper severe": 30000,
    "rear_bumper moderate": 18000,
    "rear_bumper minor": 7200,
    "front_bumper severe": 22000,
    "front_bumper moderate": 10000,
    "front_bumper minor": 3000,
    "hood severe": 14000,
    "hood moderate": 85000,
    "hood minor": 2200
}

# Calculate the total price and number of affected parts
total_price = sum(prices.get(key, 0) for key in final_results)  # calculates total price
parts_affected = len(final_results)  # counts the number of parts affected

print("Parts Affected:", parts_affected)
print("Total Price:", total_price)




/usr/local/lib/python3.10/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

Final Results: ['headlamp severe', 'hood severe', 'rear_bumper severe']
Parts Affected: 3
Total Price: 64000
